In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [ ]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

In [ ]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

In [ ]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
resnet152v2 = \
    model_development.resnet152v2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = True)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = resnet152v2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '20221003_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

In [ ]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/ResNet152v2_20221003_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)